In [1]:
import pandas as pd
import xlrd
import re

In [2]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [3]:
df=pd.read_excel(r'第三组.xlsx')
df.head()

,index,汉义,英语释义,藏（书面语）,基诺
0,1,天,sky,gnamÿnam mkha(,tsho55 na42
1,2,太阳,sun,8i ma,mj'42<33
2,3,光,light,(od,pr<44
3,4,月亮,moon,zia ba,pu33><42
4,5,星星,star,skar ma,pu33ki44


In [4]:
print(len(df))

806


In [5]:
df.dropna(axis=0,how='any',inplace=True)
print(len(df))

736


In [6]:
N=len(df)

In [7]:
re.split(r'([aeiou@]+)','daef')

['d', 'ae', 'f']

In [8]:
compound_vowel=[";i","u;","ui","ua","<u",";r","ua\\",";\\","a\\","'\\","i\\"]
for i in compound_vowel:
    print(i)

;i
u;
ui
ua
<u
;r
ua\
;\
a\
'\
i\


In [9]:
re.split(r"([iyeO;a<ou'9F]+)",'daef')

['d', 'ae', 'f']

In [10]:
re.split(r"([;i|u;|ui]+)",'daef')

['daef']

In [11]:
re.split(r"([;i|u;|ui|ua|<u|;r|ua\|;\\|a\|'\|i\\]+)",'daef')

['d', 'a', 'ef']

In [12]:
def del_zang(ls):
    split_zang=[]
    for s in ls:
        ls_biaoshu=[]
        s=str.strip(s)
        ls_yinjie=s.split(' ')
        for yinjie in ls_yinjie:
            ls_biaoshu+=re.split(r'([aeiou@]+)', yinjie)
        split_zang.append(ls_biaoshu)
    return split_zang

In [13]:
def deal_jinuo(ls):
    single_vowel=list("iyeO;a<ou'9F")
    compound_vowel=[";i","u;","ui","ua","<u",";r","ua\\",";\\","a\\","'\\","i\\"]
    
    split_jinuo=[]
    for s in ls:
        s= s.strip()
        biaoshu_jinuo=[]
        tmp1=re.split(r'([123456]+)',s) #切分声韵和声调
        tmp=[]
        for i in range(len(tmp1)):
            if len(tmp1[i].strip()):
                tmp.append(tmp1[i].strip())
            
        for i in range(0,len(tmp),2):
            tmp_jinuo=[]
            flag=1
            for vowel in compound_vowel:
                pos=tmp[i].find(vowel)
                if pos!=-1:
                    tmp_jinuo+=[tmp[i][0:pos], tmp[i][pos:]]
                    flag=0
                    break
            if flag:
                tmp_ls=re.split(r"([iyeO;a<ou'9F]+)",tmp[i])
                if len(tmp_ls[-1].strip())==0:
                    tmp_jinuo+=tmp_ls[:-1]
                else:
                    tmp_jinuo+=tmp_ls
                
            tmp_jinuo.append(tmp[i+1])
            biaoshu_jinuo+=tmp_jinuo
        split_jinuo.append(biaoshu_jinuo)
    return split_jinuo

In [14]:
deal_jinuo(['mi42 tha55 xo42','a44 sa55'])

[['m', 'i', '42', 'th', 'a', '55', 'x', 'o', '42'],
 ['', 'a', '44', 's', 'a', '55']]

In [20]:
new_table=[]

for i in range(N):
    jinuo=df['基诺'].iloc[i].split('ÿ')
    zang=df['藏（书面语）'].iloc[i].split('ÿ')
    split_zang=del_zang(zang)
    split_jinuo=deal_jinuo(jinuo)
    
    for k in range(len(jinuo)): #基诺语第k种表述
        for j in range(len(zang)): #藏语第j种表述
            tmp_line=[]
            tmp_line.append(df['index'].iloc[i])
            tmp_line.append(df['藏（书面语）'].iloc[i])
            tmp_line.append(df['基诺'].iloc[i])
            tmp_line.append(j+1)
            tmp_line.append(k+1)
            tmp_line+=split_zang[j]
            tmp_line+=['']*(15-len(split_zang[j]))
            tmp_line+=split_jinuo[k]
            tmp_line+=['']*(15-len(split_jinuo[k]))
            new_table.append(tmp_line)

In [21]:
new_table=pd.DataFrame(new_table,columns=['原index','原藏（书面语）','原基诺','藏语表述序号','基诺表述序号']+[f'藏语第{i+1}音节{j}' for i in range(5) for j in ['声母','韵母','韵尾']]+[f'基诺第{i+1}音节{j}' for i in range(5) for j in ['声母','韵母','声调']])

In [22]:
new_table.to_excel('切分结果.xlsx')

以下统计共现结果

In [43]:
df=pd.read_excel(r'切分结果无词缀借词.xlsx')
df.head(10)

,原index,原藏语,原基诺,藏语表述序号,基诺表述序号,藏语第1音节声母,藏语第1音节韵母,藏语第1音节韵尾,藏语第2音节声母,藏语第2音节韵母,藏语第2音节韵尾,藏语第3音节声母,藏语第3音节韵母,藏语第3音节韵尾,藏语第4音节声母,藏语第4音节韵母,藏语第4音节韵尾,藏语第5音节声母,藏语第5音节韵母,藏语第5音节韵尾,基诺第1音节声母,基诺第1音节韵母,基诺第1音节声调,基诺第2音节声母,基诺第2音节韵母,基诺第2音节声调,基诺第3音节声母,基诺第3音节韵母,基诺第3音节声调,基诺第4音节声母,基诺第4音节韵母,基诺第4音节声调,基诺第5音节声母,基诺第5音节韵母,基诺第5音节声调
0,1,gnamÿnam mkha(,tsho55 na42,1,1,gn,a,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tsh,o,55.0,n,a,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,gnamÿnam mkha(,tsho55 na42,2,1,n,a,m,mkh,a,(,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tsh,o,55.0,n,a,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,8i ma,mj'42<33,1,1,8,i,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mj,',42.0,NaN,<,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,(od,pr<44,1,1,(,o,d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pr,<,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,zia ba,pu33><42,1,1,z,ia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p,u,33.0,>,<,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,skar ma,pu33ki44,1,1,sk,a,r,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,p,u,33.0,k,i,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,sprin,m!33 t;33,1,1,spr,i,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,m!,33.0,t,;,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,(brug,tsho55 na42 t'44,1,1,(br,u,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tsh,o,55.0,n,a,42.0,t,',44.0,NaN,NaN,NaN,NaN,NaN,NaN
8,9,"rlu,ÿlhags pa",>i42 phj933,1,1,rl,u,",",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,>,i,42.0,phj,9,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,"rlu,ÿlhags pa",>i42 phj933,2,1,lh,a,gs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,>,i,42.0,phj,9,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
zang_shengmu=[]
for i in range(5):
    zang_shengmu+=list(df[f'藏语第{i+1}音节声母'].unique())
zang_shengmu=list(set(zang_shengmu))
zang_shengmu=[a_ for a_ in zang_shengmu if a_==a_]

In [45]:
jinuo_shengmu=[]
for i in range(5):
    jinuo_shengmu+=list(df[f'基诺第{i+1}音节声母'].unique())
jinuo_shengmu=list(set(jinuo_shengmu))
jinuo_shengmu=[a_ for a_ in jinuo_shengmu if a_==a_]

In [46]:
zang_yunwei=[]
for i in range(5):
    zang_yunwei+=list(df[f'藏语第{i+1}音节韵尾'].unique())
zang_yunwei=list(set(zang_yunwei))
zang_yunwei=[a_ for a_ in zang_yunwei if a_==a_]

In [47]:
jinuo_shengdiao=[]
for i in range(5):
    jinuo_shengdiao+=list(df[f'基诺第{i+1}音节声调'].unique())
jinuo_shengdiao=list(set(jinuo_shengdiao))
jinuo_shengdiao=[a_ for a_ in jinuo_shengdiao if a_==a_]

声母共现矩阵：藏语*基诺

In [48]:
import numpy as np
shengmu_arr=np.zeros((len(zang_shengmu),len(jinuo_shengmu)),dtype=int)

for (i,sm1) in enumerate(zang_shengmu):
    for (j,sm2) in enumerate(jinuo_shengmu):
        tmp=sum(((df['藏语第1音节声母']==sm1)|(df['藏语第2音节声母']==sm1)|(df['藏语第3音节声母']==sm1)|(df['藏语第4音节声母']==sm1)|(df['藏语第5音节声母']==sm1)) &((df['基诺第1音节声母']==sm2)|(df['基诺第2音节声母']==sm2)|(df['基诺第3音节声母']==sm2)|(df['基诺第4音节声母']==sm2)|(df['基诺第5音节声母']==sm2)))
        if tmp>1:
            shengmu_arr[i][j]=tmp

In [49]:
co_shengmu= pd.DataFrame(shengmu_arr, index=zang_shengmu, columns=jinuo_shengmu)
co_shengmu.to_excel('共现分析_以词为单位.xlsx',sheet_name='声母对应')

In [50]:
yunwei_arr=np.zeros((len(zang_yunwei),len(jinuo_shengdiao)),dtype=int)

for (i,sm1) in enumerate(zang_yunwei):
    for (j,sm2) in enumerate(jinuo_shengdiao):
        tmp=sum(((df['藏语第1音节韵尾']==sm1)|(df['藏语第2音节韵尾']==sm1)|(df['藏语第3音节韵尾']==sm1)|(df['藏语第4音节韵尾']==sm1)|(df['藏语第5音节韵尾']==sm1)) &((df['基诺第1音节声调']==sm2)|(df['基诺第2音节声调']==sm2)|(df['基诺第3音节声调']==sm2)|(df['基诺第4音节声调']==sm2)|(df['基诺第5音节声调']==sm2)))
        if tmp>1:
            yunwei_arr[i][j]=tmp

In [51]:
co_yunwei= pd.DataFrame(yunwei_arr, index=zang_yunwei, columns=jinuo_shengdiao)
writer = pd.ExcelWriter('共现分析_以词为单位.xlsx', mode="a", engine="openpyxl")
co_yunwei.to_excel(writer,sheet_name='韵尾-声调对应')
writer.close()

In [54]:
shengmu_arr=np.zeros((len(zang_shengmu),len(jinuo_shengmu)),dtype=int)

for (i,sm1) in enumerate(zang_shengmu):
    for (j,sm2) in enumerate(jinuo_shengmu):
        tmp=0
        for p in range(len(df)):
            cnt_zang=0
            cnt_jinuo=0
            for k in range(5):
                if df[f'藏语第{k+1}音节声母'].iloc[p]==sm1:
                    cnt_zang+=1
                if df[f'基诺第{k+1}音节声母'].iloc[p]==sm2:
                    cnt_jinuo+=1
            tmp+=cnt_zang*cnt_jinuo
        if tmp>1:
            shengmu_arr[i][j]=tmp
co_shengmu= pd.DataFrame(shengmu_arr, index=zang_shengmu, columns=jinuo_shengmu)
co_shengmu.to_excel('共现分析_以音节为单位.xlsx',sheet_name='声母对应')

yunwei_arr=np.zeros((len(zang_yunwei),len(jinuo_shengdiao)),dtype=int)

for (i,sm1) in enumerate(zang_yunwei):
    for (j,sm2) in enumerate(jinuo_shengdiao):
        tmp=0
        for p in range(len(df)):
            cnt_zang=0
            cnt_jinuo=0
            for k in range(5):
                if df[f'藏语第{k+1}音节韵尾'].iloc[p]==sm1:
                    cnt_zang+=1
                if df[f'基诺第{k+1}音节声调'].iloc[p]==sm2:
                    cnt_jinuo+=1
            tmp+=cnt_zang*cnt_jinuo
        if tmp>1:
            yunwei_arr[i][j]=tmp
co_yunwei= pd.DataFrame(yunwei_arr, index=zang_yunwei, columns=jinuo_shengdiao)
writer = pd.ExcelWriter('共现分析_以音节为单位.xlsx', mode="a", engine="openpyxl")
co_yunwei.to_excel(writer,sheet_name='韵尾-声调对应')
writer.close()